In [16]:
import numpy as np
import pandas as pd
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple

# Optimizacion Costo-Nutricion Dieta Vegetariana Cruda

A continuación escribe el presupuesto semanal para comprar alimentos de la dieta Vegetariana Cruda (Sin símbolo de pesos, sólo número) =:

In [17]:
presupuesto_total = float(input())

6000


Se procede a dividir el presupuesto para obtener el dinero que se destinará a las frutas, verduras y semillas oleaginosas. Con un $40 \%$ destinado a las frutas, un $40 \%$ a las semillas oleaginosas mientras que el restante $20 \%$ para las verduras.

In [18]:
def division_presupuesto(p):
    f = 0.4*p
    v = 0.2*p
    o = 0.4*p
    return f,v,o
W_f, W_v, W_o = division_presupuesto(presupuesto_total)

## Algoritmo Evolutivo para Knapsack 0/1 Problem

In [10]:
Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]


def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)


def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]


def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(gene) for gene in population],
        k=2
    )


def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i



## Algoritmo Greedy para Fractional Knapsack

In [9]:
# Un algoritmo greedy para el problema de knapsack fraccional
# Notemos que se ordena por v/w si que modifiquemos v o w, para que se pueda
# dar la salida con los indices que se quedaran en el knapsack al final

def KnapsackFrac(v, w, W):
    order = bubblesortByRatio(v, w)            
    weight = 0.0                               
    value = 0.0                                
    knapsack = []                              
    n = len(v)
    index = 0                                   
    while (weight < W) and (index < n):
        if weight + w[order[index]] <= W:        
            knapsack.append((order[index], 1.0))   
            weight = weight + w[order[index]]
            value = value + v[order[index]]
        else:
            fraction = (W - weight) / w[order[index]]  
            knapsack.append((order[index], fraction))
            weight = W
            value = value + v[order[index]] * fraction
        index = index + 1
    return (knapsack, value)                       
  
# Ordenamos la lista en orden descendente por la razon de list1[1] en list2[i]
# pero en vez de re-ordenar list1 y list2, guardamos el orden en 
# un arreglo a parte.
def bubblesortByRatio(list1, list2):
    n = len(list1)
    order = list(range(n))
    for i in range(n - 1, 0, -1):     
        for j in range(0, i):           
            if ((1.0 * list1[order[j]]) / list2[order[j]]) < ((1.0 * list1[order[j+1]]) / list2[order[j+1]]):
                temp = order[j]              
                order[j] = order[j+1]
                order[j+1] = temp
    return order

w = [65,180,295,52,120,135,60,45,60,250,400,500,400,130]
v = [5,4,3,3,3,3,5,4,5,5,4,4,4,4]
W = 60
print(KnapsackFrac(v, w, W)[0])

[(7, 1.0), (6, 0.25)]
